In [ ]:
import pandas as pd
import geopandas as gpd
from pathlib import Path
import folium

In [ ]:
EXPORT = False

In [ ]:
RAW_DATA_PATH = Path.cwd().parent / "raw_data"
CLEAN_DATA_PATH = Path.cwd().parent / "src" / "tripmap"

In [ ]:
test_df = pd.read_csv(RAW_DATA_PATH / "points.csv")
test_df["date"] = pd.to_datetime(test_df["date"], format="%d-%m-%Y")
test_df = test_df.set_index("date")

trip = gpd.GeoDataFrame(
    test_df,
    geometry=gpd.points_from_xy(test_df["longitude"], test_df["latitude"]),
    crs="EPSG:4326",
)

display(trip)

In [ ]:
collector = []
for date in test_df.index:
    file = RAW_DATA_PATH / f"{date.date()}.gpx"
    if not file.exists():
        continue
    daily_route = gpd.read_file(file, layer="tracks")
    daily_route["date"] = date
    daily_route["Descrizione"] = "Treno"
    daily_route = daily_route[["date", "geometry", "Descrizione"]].set_index("date")
    collector.append(daily_route)

all_routes = pd.concat(collector)

In [ ]:
color_palette = [
    "#058240",
    "#AA1803",
    "#87C159",
    "#048399",
    "#F3C301",
    "#85c3fe",
]

In [ ]:
trip = gpd.GeoDataFrame(
    pd.concat([trip, all_routes]), geometry="geometry", crs="EPSG:4326"
).reset_index()
trip["Giorno"] = (trip["date"].dt.day - 7).astype(int)
trip = trip.set_index("date").sort_index()
trip["color"] = [
    color_palette[day % len(color_palette)] for day in trip["Giorno"].to_list()
]

display(trip)

In [ ]:
trip.explore(
    color=trip["color"].to_list(),
    tiles="CartoDB positron",
    tooltip=False,
    popup=["Giorno", "Descrizione"],
    style_kwds=dict(
        style_function=lambda x: {
            "html": f"""<span   class="fa-solid fa-location-dot" 
                                style="color:{x["properties"]["color"]};
                                font-size:30px"></span>"""
        },
    ),
    marker_type="marker",
    marker_kwds=dict(icon=folium.DivIcon()),
)

In [ ]:
if EXPORT:
    trip[trip["latitude"].isna()].reset_index().to_file(
        RAW_DATA_PATH / "routes.json", driver="GeoJSON"
    )
    trip[~trip["latitude"].isna()].reset_index().to_file(
        RAW_DATA_PATH / "points.json", driver="GeoJSON"
    )

___